# HMM 

An HMM Part-of-Speech (POS) Tagger is a statistical model that assigns parts of speech to words in a sentence using the Hidden Markov Model (HMM) framework.

First we import our model with the needed dependencies:

In [ ]:
import numpy as np
from collections import defaultdict, Counter
from conllu_dataloader import *
from model.hmm import HMMPOSTagger

When we initialize the model we have these parameters: 

- `tags`: set of possible tags
- `vocab`: set of possible words

 

In [ ]:
hmm = HMMPOSTagger(tags, vocabulary)

After initializing we get the following attributes:

- `self.tags2idx` = dictionary to get the idx from the tags
- `self.idx2tags` = dictionary to get the tags from the idx
- `self.tags` = set of used tags
- `self.Q`= number of tags
- `self.vocab` = vocabulary
- `self.transition_counts` = matrix to store the transition counts
- `self.emission_counts` = matrix to store the emission counts
- `self.transition_probs` = matrix to store the transition probabilities
- `self.emission_probs` = matrix to store the emission probabilities
- `self.word_counts` = counter of each word
- `self.tag_counts` = counter of each tag

# CONLLU DATALOADER

Converts conllu files to csv. It generates 4 files: sentences, sentences post tags, general tags, vocabulary

# Loader class: functions

## TRAIN FUNCTION

Function to train our HMM model

Our train function has the following parameters:
- `sentences`
list of all the sentences in our train dataset
- `pos_tags`
list of all the PoS tags of the sentences in our train dataset
- `change_vocab` if true, it updates the vocab by removing the least frequent words and turning them all to <UNK>

First the model counts the occurrences of the tags and the words and stores them in two dictionaries. 
The first one is "transition_counts" where we store each tags that follows another tag; and the second one is "emission_counts" where we store each word associated with a tag.
We also take into account the number of occurrences of each tag and word, and we store them in "tag_counts" and "word_counts" respectively.

Every time we start a new sentence we insert a "*" in the first position and a <STOP> in the last position. This is to take into consideration the first and last words of a sentence. 

After running all the sentences we check if we have to change the vocabulary. If so, we revise the words in dictionary "word_counts", we remove the words that appear less than 5 times and we add the counts to the <UNK> token. We do this so that the model can work better with unknown words.

Then, to sample with the model, we have to get the probabilities of transitions and emission counts. This is achieved with the following function:
$$
P(q_i|q_{i-1})={count(q_{i-1},q_i) \over count(q_{i-1})}
$$

Where $ q_{i-1} $ are going to be the previous tags and $ q_i $ the current tags in transition counts, and with emission counts, the $ q_{i-1} $ are going to be the current words and the $q_i$ are going to be the current tags. 


After running the train function we will have two matrices:
 1) transition probabilty matrix
 2) emission probabilty matrix


In [ ]:
hmm.train(sentences, pos_tags, change_vocab = True)

If we want to inspect the transition and emission probabilities we can check them like this:

In [ ]:
print(hmm.transition_probs)
print(hmm.emission_probs)

# VITERBI ALGORITHM

This algorithm determines what PoS tags a sentence contains using the HMM that we have trained. 
The parameter that we use here is:
- `sentence`: the sentence we have to predict the PoS tags of


First, we take a look at the sentence and we search each word in the vocabulary. If there is no such word in the vocabulary we replace it with the <UNK> token.
Next, we apply the algorithm. To do that, we define two variables: the viterbi matrix, where we will store the probabilities; and the backpointer, where we store the most probable path. At this point we calculate the probabilty of the word for each tag with the following function: $$viterbi[q,t]=\max viterbi[q',t-1]*A_{[q',q]}*B_{[q,t]}$$

Where $A$ is the transition matrix and $B$ the mission matrix.

The $q'$ that gets the maximum pobabilty is stored in the backpointer. This way the viterbi function gives back the  initial sentence and the PoS tags predicted by the algorithm.

In [ ]:
sentence = ['Jeremy','loves','NLP']
hmm.viterbi_alg(sentence)

# EVALUATE FUNCTION

This function evaluates the test dataset with Viterbi and, with whatever Viterbi returns, we calculate accuracy and F1.

In [ ]:
test = [['Jeremy', 'Loves', 'NLP'],
        ['How', 'do', 'people', 'live', 'in', 'houses'],
        ['Those','in', 'power', 'have', 'little', 'interest', 'in', 'education']
]
tags = [['NOUN', 'VERB', 'NOUN'],
        ['ADV', 'VERB', 'NOUN', 'VERB', 'ADP', 'NOUN'],
        ['PRON', 'ADP', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'ADP', 'NOUN']]
        
hmm.evaluate(test, tags)